In [4]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
import MathOptInterface as MOI
import Ipopt
import FiniteDiff
import ForwardDiff as FD
import Convex as cvx
import ECOS
using LinearAlgebra
using Plots
using Random
using JLD2
using Test
using MeshCat
const mc = MeshCat
using StaticArrays
using Printf

include(joinpath(@__DIR__, "utils", "fmincon.jl"))
include(joinpath(@__DIR__, "utils", "quaternions.jl"))
include(joinpath(@__DIR__, "utils", "util.jl"))

  Activating environment at `~/Desktop/16745/recitations-2024/16745proj/Project.toml`


create_idx (generic function with 1 method)

In [6]:
#--------PLACEHOLDER
tf = 10
dt = 0.1
t_vec = 0:dt:tf
N = length(t_vec)
#-----------------

L = NUM_QUAD = 4
nx = 13 * L + 6
nu = 6 * L + L # 5 true controls + 'slack' variable to normalize quat
idx = create_idx(nx, nu, N)

(nx = 58, nu = 24, N = 45, nz = 3666, nc = 2552, x = UnitRange{Int64}[1:58, 83:140, 165:222, 247:304, 329:386, 411:468, 493:550, 575:632, 657:714, 739:796  …  2871:2928, 2953:3010, 3035:3092, 3117:3174, 3199:3256, 3281:3338, 3363:3420, 3445:3502, 3527:3584, 3609:3666], u = UnitRange{Int64}[59:82, 141:164, 223:246, 305:328, 387:410, 469:492, 551:574, 633:656, 715:738, 797:820  …  2847:2870, 2929:2952, 3011:3034, 3093:3116, 3175:3198, 3257:3280, 3339:3362, 3421:3444, 3503:3526, 3585:3608], c = UnitRange{Int64}[1:58, 59:116, 117:174, 175:232, 233:290, 291:348, 349:406, 407:464, 465:522, 523:580  …  1973:2030, 2031:2088, 2089:2146, 2147:2204, 2205:2262, 2263:2320, 2321:2378, 2379:2436, 2437:2494, 2495:2552])

In [21]:
function quad_costs(n_lift, m_lift, scenario=:p2pa)
    if scenario == :hover
        q_diag = 10.0 * ones(n_lift)
        q_diag[4:7] .= 1e-6

        r_diag = 1.0e-3 * ones(m_lift)
        r_diag[end] = 1

        qf_diag = copy(q_diag) * 10.0
    elseif scenario == :p2pa
        q_diag = 1.0 * ones(n_lift)
        q_diag[1] = 1e-5
        # q_diag[4:7] .*= 25.0
        # q_diag

        r_diag = 1.0e-3 * ones(m_lift)
        # r_diag = 1.0e-3*ones(m_lift)
        r_diag[end] = 1

        qf_diag = 100 * ones(n_lift)
    else
        q_diag = 1e-1 * ones(n_lift)
        q_diag[1] = 1e-3
        q_diag[4:7] .*= 25.0

        r_diag = 2.0e-3 * ones(m_lift)

        # r_diag = 1.0e-3*ones(m_lift)
        r_diag[end] = 1

        qf_diag = 100 * ones(n_lift)
    end
    return q_diag, r_diag, qf_diag
end

function load_costs(n_load, m_load, scenario=:p2pa)
    if scenario == :hover
        q_diag = 10.0 * ones(n_load) #

        r_diag = 1 * ones(m_load)
        qf_diag = 10.0 * ones(n_load)
    elseif scenario == :p2p
        q_diag = 1.0 * ones(n_load) #

        # q_diag = 0*ones(n_load)
        q_diag[1] = 1.0e-5
        r_diag = 1 * ones(m_load)
        qf_diag = 0.0 * ones(n_load)

        # q_diag = 1000.0*ones(n_load)
        # r_diag = 1*ones(m_load)
        # qf_diag = 1000.0*ones(n_load)
    else
        q_diag = 0.5e-1 * ones(n_load) #

        # q_diag = 0*ones(n_load)
        # q_diag[1] = 1e-3
        r_diag = 1 * ones(m_load)
        qf_diag = 0.0 * ones(n_load)

        # q_diag = 1000.0*ones(n_load)
        # r_diag = 1*ones(m_load)
        # qf_diag = 1000.0*ones(n_load)

    end
    return q_diag, r_diag, qf_diag
end


load_costs (generic function with 2 methods)

In [16]:
function quad_cost(params::NamedTuple, Z::Vector)::Real
    idx, N = params.idx, params.N
    Q, R, Qf = params.Q, params.R, params.Qf

    #xg should be a stacked vector of goals [x1g;x2g;x3g...]
    xg = params.xg


    #LQR cost 
    J = 0
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]

        J += 0.5 * (xi - xg)' * Q * (xi - xg)
        J += 0.5 * ui' * R * ui

    end

    # dont forget terminal cost 
    J += 0.5 * (Z[idx.x[N]] - xg)' * Q * (Z[idx.x[N]] - xg)

    return J
end

#To add to params/model:
n_lift = 13
m_lift = 6 #5 + 1 for quaternion slack variable 
n_load = 6
m_load = NUM_QUAD # num quad /L

quad_cost (generic function with 1 method)

In [24]:
params = (
    model=model,
    nx=nx,
    nu=nu,
    tf=tf,
    dt=dt,
    t_vec=t_vec,
    N=N,
    M1=M1,
    M2=M2,
    J1=J1,
    J2=J2,
    xic=xic,
    xg=xg,
    idx=idx,
    Q=Q, R=R, Qf=Qf,
    Xref=Xref,
    Uref=Uref,
    n_lift=13,
    m_lift=6, #5 + 1 for quaternion slack variable 
    n_load=6,
    m_load=NUM_QUAD # num quad /L
)
model = (lift=quad_params, load=load_params)

function quad_cost(params::NamedTuple, Z::Vector)::Real
    # cost function 
    idx, N, xg = params.idx, params.N, params.xg
    Q, R, Qf = params.Q, params.R, params.Qf
    Xref, Uref = params.Xref, params.Uref

    # TODO: input walker LQR cost 

    J = 0
    for i = 1:N-1
        J += 0.5 * (Z[idx.x[i]] - Xref[i])' * Q * (Z[idx.x[i]] - Xref[i]) +
             0.5 * (Z[idx.u[i]] - Uref[i])' * R * (Z[idx.u[i]] - Uref[i])
    end
    J += 0.5 * (Z[idx.x[N]] - Xref[N])' * Qf * (Z[idx.x[N]] - Xref[N])


    return J
end

function dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    M1, M2 = params.M1, params.M2
    J1, J2 = params.J1, params.J2
    model = params.model

    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), idx.nc)

    for i = 1:N-1
        #3
        xk = Z[idx.x[i]]
        uk = Z[idx.u[i]]
        c[idx.c[i]] = Z[idx.x[i+1]] - rk4(model, combined_system_dynamics, xk, uk, dt)
    end


    return c
end

UndefVarError: UndefVarError: model not defined

In [ ]:
function quad_obstacle_proximity_constraints(params::NamedTuple, Z::Vector, obstacle_positions::Vector{Vector}):: Vector
    idx, N, dt = params.idx, params.N, params.dt
    NUM_QUAD = params.NUM_QUAD
    #Assuming obstacle positions is of form [[x1,y1],[x2,y2]]
    for i=1:N: #Each timestep
        xi = Z[idx.x[i]]
        xi_quads = [xi[(quad-1)*13 + 1:13*quad] for quad=1:NUM_QUAD]
        

    end
end

function inequality_constraints(params::NamedTuple, Z)
end